<h3>Ankit Bansal Complex SQL Query 1 | Derive Points table for ICC tournament</h3> 

<h2> query data set </h2>

In [0]:
%sql

create table icc_world_cup
(
Team_1 Varchar(20),
Team_2 Varchar(20),
Winner Varchar(20)
);
INSERT INTO icc_world_cup values('India','SL','India');
INSERT INTO icc_world_cup values('SL','Aus','Aus');
INSERT INTO icc_world_cup values('SA','Eng','Eng');
INSERT INTO icc_world_cup values('Eng','NZ','NZ');
INSERT INTO icc_world_cup values('Aus','India','India');

num_affected_rows,num_inserted_rows
1,1


<h2> Solution in sql(hive) </h2>

In [0]:
%sql
with cte1 as (
  select team_1 as team , sum(tcnt) as matches_played from (
    select team_1,count(*)as tcnt from icc_world_cup group by team_1
    union all
    select team_2,count(*)as tcnt from icc_world_cup group by team_2
    )a1
  group by team_1
  )

select team,matches_played,nvl(wn,0) as no_of_win,matches_played-nvl(wn,0)as no_of_losses from cte1 
left join (
  select Winner,count(*)as wn from icc_world_cup group by Winner
  )a2 on a2.Winner=cte1.team

team,matches_played,no_of_win,no_of_losses
SL,2,0,2
Eng,2,1,1
India,2,2,0
SA,1,0,1
Aus,2,1,1
NZ,1,1,0


<h2>Solution in Pyspark </h2>

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import sum

In [0]:
# put data in pyspark data frame from the hive table
df=spark.sql("select * from icc_world_cup")

In [0]:
df1=df.groupBy("team_1").count()
df2=df.groupBy("team_2").count()
df_union=df1.unionAll(df2)
df_matchs=df_union.groupBy("team_1").agg(sum("count").alias("matches_played"))
df_win=df.groupBy("Winner").count()
df_no_of_win=df_matchs.join(df_win,df_matchs.team_1==df_win.Winner,"left").\
    select(df_matchs["team_1"].alias("Team"),"matches_played",df_win["count"].alias("no_of_win")).fillna(0)
df_final=df_no_of_win.withColumn("no_of_losses",df_no_of_win.matches_played-df_no_of_win.no_of_win)
df_final.show()

+-----+--------------+---------+------------+
| Team|matches_played|no_of_win|no_of_losses|
+-----+--------------+---------+------------+
|   SL|             2|        0|           2|
|  Eng|             2|        1|           1|
|India|             2|        2|           0|
|   SA|             1|        0|           1|
|  Aus|             2|        1|           1|
|   NZ|             1|        1|           0|
+-----+--------------+---------+------------+

